In [0]:
events = spark.sql("SELECT * FROM events_tbl")

In [0]:
events.describe(["price"]).show()


In [0]:
# Hypothesis: weekday vs weekend conversion

from pyspark.sql import functions as F
weekday = events.withColumn("is_weekend",
    F.dayofweek("event_time").isin([1,7]))
weekday.groupBy("is_weekend", "event_type").count().show()

In [0]:
from pyspark.sql.window import Window

features = events.withColumn("hour", F.hour("event_time")) \
    .withColumn("day_of_week", F.dayofweek("event_time")) \
    .withColumn("price_log", F.log(F.col("price")+1)) \
    .withColumn("time_since_first_view",
        F.unix_timestamp("event_time") -
        F.first("event_time").over(Window.partitionBy("user_id").orderBy("event_time")))
